# 1. Preparation

### Preparation

In [1]:
# Import modules
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# 2. Define Models

### Define a Neural Network Model that Dropout Applied

In [2]:
# Define a class
class DropoutNet(nn.Module):
    # Initialize the class
    def __init__(self):
        super(DropoutNet, self).__init__()
        # Convert 28x28 input data to 1-Dimensional data
        self.fc1 = nn.Linear(784, 500)  # '784': 28x28
        
        # Apply dropout
        self.dropout = nn.Dropout(p = 0.5)  # 'p=0.5': 50%
        
        # Fully connected layer 2: Linear
        self.fc2 = nn.Linear(500, 10)
        
    # Forward
    def forward(self, x):
        # Resize input data: Flatten data to 1-Dimensional data
        x = x.view(x.size(0), -1)
        
        # Apply ReLU to 'fc1'
        x = torch.relu(self.fc1(x))
        
        # Apply Dropout to 'fc1'
        x = self.dropout(x)
        
        # Run 'fc2'
        x = self.fc2(x)
        
        return x
        

### Define a Neural Network Model without Dropout

In [3]:
# Define a class
class NonDropoutNet(nn.Module):
    # Initialize the class
    def __init__(self):
        super(NonDropoutNet, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)

    # Forward
    def forward(self, x):
        # Convert 28x28 input data to 1-Dimensional data
        x = x.view(x.size(0), -1)
        
        # Apply ReLU
        x = torch.relu(self.fc1(x))
    
        # Run 'fc2'
        x = self.fc2(x)
        
        return x

# 3. Set Data

### Load Dataset with Pre-Processing

In [4]:
# Set transforms
train_transform = transforms.Compose([
    transforms.AutoAugment(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.3,))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.3, ))
])

# Load dataset
train_dataset = datasets.MNIST(root = './data/0706-MNIST',
                               train = True, 
                               download = True,
                               transform = train_transform)
test_dataset = datasets.MNIST(root = './data/0706-MNIST',
                              train = False,
                              download = True,
                              transform = test_transform)

### Set DataLoaders

In [5]:
# Set dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size = 64,
                                           shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size = 64,
                                          shuffle = False)

# 4. Set a Model with Dropout 

### Initialize the Model with Dropout

In [6]:
# Initialize models
dropout_model = DropoutNet()
print(dropout_model)

DropoutNet(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


### Set Loss Function and Optimizer

In [7]:
# Set Loss Function
dropout_criterion = nn.CrossEntropyLoss()
print(dropout_criterion)

# Set Optimizer
dropout_optimizer = optim.SGD(dropout_model.parameters(), lr=0.001)
print(dropout_optimizer)

CrossEntropyLoss()
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0
    nesterov: False
    weight_decay: 0
)


### Fit the Model with Dropout

In [8]:
# Fit the Model with dropout
for epoch in range(10):
    # Declare Train mode
    dropout_model.train()
    
    # Fit the model
    for images, labels in train_loader:
        dropout_optimizer.zero_grad() # Initialize gradients of all model parameters
        dropout_output = dropout_model(images)
        dropout_loss = dropout_criterion(dropout_output, labels)
        dropout_loss.backward()
        dropout_optimizer.step()
        
# Declare Test mode
dropout_model.eval()

# Fit models
with torch.no_grad():
    # Initialize parameters
    dropout_correct = 0
    dropout_total = 0
    
    for images, labels in test_loader:
        test_out = dropout_model(images)
        _, dropout_pre = torch.max(test_out.data, 1)
        dropout_total += images.size(0)
        dropout_correct += (dropout_pre == labels).sum().item()
        
    print('Accuracy of Model that Dropout applied: {:.2f}%'.format(100 * dropout_correct / dropout_total) )

Accuracy of Model that Dropout applied: 90.10%


# 5. Set a Model without Dropout

### Initialize the Model without Dropout

In [9]:
# Initialize models
non_dropout_model = NonDropoutNet()
print(non_dropout_model)

NonDropoutNet(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


### Set Loss Function and Optimizer

In [10]:
# Set Loss Function
non_dropout_criterion = nn.CrossEntropyLoss()
print(non_dropout_criterion)

# Set Optimizer
non_dropout_optimizer = optim.SGD(non_dropout_model.parameters(), lr=0.001)
print(non_dropout_optimizer)

CrossEntropyLoss()
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0
    nesterov: False
    weight_decay: 0
)


### Fit the Model without Dropout

In [11]:
# Fit the Model with dropout
for epoch in range(10):
    # Declare Train mode
    non_dropout_model.train()
    
    # Fit the model
    for images, labels in train_loader:
        non_dropout_optimizer.zero_grad()
        non_dropout_output = non_dropout_model(images)
        non_dropout_loss = non_dropout_criterion(non_dropout_output, labels)
        non_dropout_loss.backward()
        non_dropout_optimizer.step()
        
# Declare Test mode
non_dropout_model.eval()

# Fit models
with torch.no_grad():
    # Initialize parameters
    non_dropout_correct = 0
    non_dropout_total = 0
    
    for images, labels in test_loader:
        test_out = non_dropout_model(images)
        _, non_dropout_prediction = torch.max(test_out.data, 1)
        non_dropout_total += images.size(0)
        non_dropout_correct += (non_dropout_prediction == labels).sum().item()
        
    print('Accuracy of Model that Dropout NOT applied: {:.2f}%'.format(100 * non_dropout_correct / non_dropout_total) )

Accuracy of Model that Dropout NOT applied: 89.92%
